This is the optimization code for May's algorithm with depth-4, depth-5 and depth-6. 
To execute the code we have to execute the below two cells first.

In [12]:
import collections
import random
from random import  uniform
from math import *
import scipy.optimize as opt
import math 
import matplotlib.pyplot as plt

def H(c):
    """
    Entropy function
    """
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    """
    binomial coefficient
    """
    # if k/n not in ZZ:
    #     return -100
    if(n<=0):
        return 0.
    return n * H(k/n)

def multiH(n,c):
    """
    multinomial coefficient
    """
    if sum(c)>n:
        return 0
    tot=0
    val=n
    for i in c:
        tot+=binomH(n,i)
        n-=i
    return tot


def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

def r(x,y,z):
    return [(ru(x,y)) for i in range(z)]

In [13]:
def reps1(w1,w0,omid,zmid2,zmid1,p): 
   #length of whole vec = p 
    if w1 <= 0.000001 or p == 0.:
        return 0
#     if p < w1 or p - 2*w1 < 2*zmid1:
#         return 0.
    
    return 2*multiH(w1, [omid,omid,w1*.5-omid]) + multiH(p-2*w1,[zmid2,zmid2,zmid1,zmid1])


def reps2(n1,n0,n2,tbar,o,z2,z1,p): 
    '''
    
    We represent a vector of length p having n1 many pm1,
    n2 many pm2 and rest zero, using rep-2 representation
    from our CRYPTO 2023 submission.
    
    '''
    
    if n1 < 0.000001 or p == 0.:
        return 0.
#     if p < n1 or p<n2 or p - 2*n1 - n2 < 2*z1:
#         return 0.
    
    return 2*multiH(n2,[tbar,tbar])+2*multiH(n1, [o, o, n1*.5-o])+multiH(n0, [z2, z2, z1, z1])

Depth-4

In [17]:
set_vars = collections.namedtuple('LWE', ' omid zmid2 zmid1 tbar_1 o_1 z2_1 z1_1 tbar_2 o_2 z2_2 z1_2 ')
'''
tbar1  = (n2_1-t)/2
This set of variables can be further increased to 'tbar_3 o_3 z2_3 z1_3' to accomodate
more depth.
'''
def lwe(f) : return wrap(f, set_vars)



n2_0 = lambda x : 0
n1_0 = lambda x : w/2
n0_0 = lambda x : 1 - 2*n1_0(x)

n2_1 = lambda x : x.omid+x.zmid2              
n1_1 = lambda x : n1_0(x)/2+x.zmid1           
n0_1 = lambda x : 1-n1_0(x)-2*(x.zmid1+x.omid+x.zmid2)   

n2_2 = lambda x : x.tbar_1+x.o_1+x.z2_1
n1_2 = lambda x : n1_1(x)/2+x.z1_1+n2_1(x)-2*x.tbar_1
n0_2 = lambda x : 1-n1_1(x)-2*n2_1(x)+2*(x.tbar_1-x.z1_1-x.o_1-x.z2_1)

n2_3 = lambda x : x.tbar_2+x.o_2+x.z2_2
n1_3 = lambda x : n1_2(x)/2+x.z1_2+n2_2(x)-2*x.tbar_2
n0_3 = lambda x : 1-n1_2(x)-2*n2_2(x)+2*(x.tbar_2-x.z1_2-x.o_2-x.z2_2)




R_1 = lambda x : reps1(n1_0(x),n0_0(x),x.omid,x.zmid2,x.zmid1,1)
R_2 = lambda x : reps2(n1_1(x),n0_1(x),n2_1(x),x.tbar_1,x.o_1,x.z2_1,x.z1_1,1)
R_3 = lambda x : reps2(n1_2(x),n0_2(x),n2_2(x),x.tbar_2,x.o_2,x.z2_2,x.z1_2,1)


S_1 = lambda x : multiH(1,[n2_1(x),n2_1(x),n1_1(x),n1_1(x)])
S_2 = lambda x : multiH(1,[n2_2(x),n2_2(x),n1_2(x),n1_2(x)])
S_3 = lambda x : multiH(1,[n2_3(x),n2_3(x),n1_3(x),n1_3(x)])


L_1 = lambda x : S_1(x)-R_1(x)
L_2 = lambda x : S_2(x)-R_2(x)
L_3 = lambda x : S_3(x)-R_3(x)
L_4 = lambda x : S_3(x)/2

T_4 = lambda x : L_4(x)
T_3 = lambda x : 2*L_4(x) - R_3(x)
T_2 = lambda x : 2*L_3(x) - R_2(x) + R_3(x)
T_1 = lambda x : 2*L_2(x) - R_1(x) + R_2(x)
T_0 = lambda x : 2*L_1(x) - 1


def time(x):
    x=set_vars(*x)
    max_of_T_i = max(T_4(x),T_3(x),T_2(x),T_1(x),T_0(x))
    return max_of_T_i

def memory(x):
    x=set_vars(*x)
    max_of_L_i = max(L_4(x),L_3(x),L_2(x),L_1(x))
    return max_of_L_i

constraints_lwe = [
    
    
   # { 'type' : 'ineq',   'fun' : lwe(lambda x : time(x)-memory(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_3(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_3(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_3(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_3(x))},
    
     
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_4(x))},
    
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_4(x))},
#     { 'type' : 'ineq',   'fun' : lwe(lambda x : T_0(x))},  # this is negative
    
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_0(x)*.5-x.omid)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+n0_0(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_1+n2_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_1+n1_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_1-x.z1_1+n0_1(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_2+n2_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_2+n1_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_2-x.z1_2+n0_2(x)/2)},
    
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x))},    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x)-R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x)-R_2(x))},
    

]

F4 = []
    
for weight in range(25,26):
    w = weight *.02
    res = 100
    ress = 5000
    for j in range(90):
        start=[uniform(0,.009)]*11
        
        bounds=[(0.,0.1)]*11
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
               res = r
               ress = result
    if(res<memory(ress.x)):
        print('Memory exceeds time')
    y = set_vars(*(ress.x))
    print((round(w,4),round(res,4)))
    #print(ress.x)
    F4.append((round(w,4),round(res,4)))

(0.5, 0.3447)


Depth-5

In [24]:
set_vars = collections.namedtuple('LWE', ' omid zmid2 zmid1 tbar_1 o_1 z2_1 z1_1 tbar_2 o_2 z2_2 z1_2 tbar_3 o_3 z2_3 z1_3 ')
'''
tbar1  = (n2_1-t)/2
This set of variables can be further increased to 'tbar_3 o_3 z2_3 z1_3' to accomodate
more depth.
'''
def lwe(f) : return wrap(f, set_vars)



n2_0 = lambda x : 0
n1_0 = lambda x : w/2
n0_0 = lambda x : 1 - 2*n1_0(x)

n2_1 = lambda x : x.omid+x.zmid2              
n1_1 = lambda x : n1_0(x)/2+x.zmid1           
n0_1 = lambda x : 1-n1_0(x)-2*(x.zmid1+x.omid+x.zmid2)   

n2_2 = lambda x : x.tbar_1+x.o_1+x.z2_1
n1_2 = lambda x : n1_1(x)/2+x.z1_1+n2_1(x)-2*x.tbar_1
n0_2 = lambda x : 1-n1_1(x)-2*n2_1(x)+2*(x.tbar_1-x.z1_1-x.o_1-x.z2_1)

n2_3 = lambda x : x.tbar_2+x.o_2+x.z2_2
n1_3 = lambda x : n1_2(x)/2+x.z1_2+n2_2(x)-2*x.tbar_2
n0_3 = lambda x : 1-n1_2(x)-2*n2_2(x)+2*(x.tbar_2-x.z1_2-x.o_2-x.z2_2)

n2_4 = lambda x : x.tbar_3+x.o_3+x.z2_3
n1_4 = lambda x : n1_3(x)/2+x.z1_3+n2_3(x)-2*x.tbar_3
n0_4 = lambda x : 1-n1_3(x)-2*n2_3(x)+2*(x.tbar_3-x.z1_3-x.o_3-x.z2_3)



R_1 = lambda x : reps1(n1_0(x),n0_0(x),x.omid,x.zmid2,x.zmid1,1)
R_2 = lambda x : reps2(n1_1(x),n0_1(x),n2_1(x),x.tbar_1,x.o_1,x.z2_1,x.z1_1,1)
R_3 = lambda x : reps2(n1_2(x),n0_2(x),n2_2(x),x.tbar_2,x.o_2,x.z2_2,x.z1_2,1)
R_4 = lambda x : reps2(n1_3(x),n0_3(x),n2_3(x),x.tbar_3,x.o_3,x.z2_3,x.z1_3,1)


S_1 = lambda x : multiH(1,[n2_1(x),n2_1(x),n1_1(x),n1_1(x)])
S_2 = lambda x : multiH(1,[n2_2(x),n2_2(x),n1_2(x),n1_2(x)])
S_3 = lambda x : multiH(1,[n2_3(x),n2_3(x),n1_3(x),n1_3(x)])
S_4 = lambda x : multiH(1,[n2_4(x),n2_4(x),n1_4(x),n1_4(x)])


L_1 = lambda x : S_1(x)-R_1(x)
L_2 = lambda x : S_2(x)-R_2(x)
L_3 = lambda x : S_3(x)-R_3(x)
L_4 = lambda x : S_4(x)-R_4(x)
L_5 = lambda x : S_4(x)/2

T_5 = lambda x : L_5(x)
T_4 = lambda x : 2*L_5(x) - R_4(x)
T_3 = lambda x : 2*L_4(x) - R_3(x) + R_4(x)   # missed R_(4)(x) in this line
T_2 = lambda x : 2*L_3(x) - R_2(x) + R_3(x)
T_1 = lambda x : 2*L_2(x) - R_1(x) + R_2(x)
T_0 = lambda x : 2*L_1(x) - 1


def time(x):
    x=set_vars(*x)
    max_of_T_i = max(T_5(x),T_4(x),T_3(x),T_2(x),T_1(x),T_0(x))
    return max_of_T_i

def memory(x):
    x=set_vars(*x)
    max_of_L_i = max(L_5(x),L_4(x),L_3(x),L_2(x),L_1(x))
    return max_of_L_i

constraints_lwe = [
    
    
   # { 'type' : 'ineq',   'fun' : lwe(lambda x : time(x)-memory(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_4(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_4(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_4(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_4(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_4(x))},

     
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_5(x))},


    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_5(x))},
#     { 'type' : 'ineq',   'fun' : lwe(lambda x : T_0(x))},  # this is negative

    
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_0(x)*.5-x.omid)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+n0_0(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_1+n2_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_1+n1_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_1-x.z1_1+n0_1(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_2+n2_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_2+n1_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_2-x.z1_2+n0_2(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_3+n2_3(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_3+n1_3(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_3-x.z1_3+n0_3(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_4(x))},    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x)-R_4(x))},    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x)-R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x)-R_2(x))},
    

]

F5 = []    
for weight in range(25,26):
    w = weight*.02
    res = 100
    ress = 5000
    for j in range(30):
        start=[uniform(0,.009)]*15
        
        bounds=[(0.,0.1)]*15
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
               res = r
               ress = result
    if(res<memory(ress.x)):
        print('Memory exceeds time')
    print((round(w,4),round(res,4)))  
    F5.append((round(w,4),round(res,4)))

(0.5, 0.3294)


Depth-6

In [19]:
set_vars = collections.namedtuple('LWE', ' omid zmid2 zmid1 tbar_1 o_1 z2_1 z1_1 tbar_2 o_2 z2_2 z1_2 tbar_3 o_3 z2_3 z1_3 tbar_4 o_4 z2_4 z1_4 ')
'''
tbar1  = (n2_1-t)/2
This set of variables can be further increased to 'tbar_3 o_3 z2_3 z1_3' to accomodate
more depth.
'''
def lwe(f) : return wrap(f, set_vars)



n2_0 = lambda x : 0
n1_0 = lambda x : w/2
n0_0 = lambda x : 1 - 2*n1_0(x)

n2_1 = lambda x : x.omid+x.zmid2              
n1_1 = lambda x : n1_0(x)/2+x.zmid1           
n0_1 = lambda x : 1-n1_0(x)-2*(x.zmid1+x.omid+x.zmid2)   

n2_2 = lambda x : x.tbar_1+x.o_1+x.z2_1
n1_2 = lambda x : n1_1(x)/2+x.z1_1+n2_1(x)-2*x.tbar_1
n0_2 = lambda x : 1-n1_1(x)-2*n2_1(x)+2*(x.tbar_1-x.z1_1-x.o_1-x.z2_1)

n2_3 = lambda x : x.tbar_2+x.o_2+x.z2_2
n1_3 = lambda x : n1_2(x)/2+x.z1_2+n2_2(x)-2*x.tbar_2
n0_3 = lambda x : 1-n1_2(x)-2*n2_2(x)+2*(x.tbar_2-x.z1_2-x.o_2-x.z2_2)

n2_4 = lambda x : x.tbar_3+x.o_3+x.z2_3
n1_4 = lambda x : n1_3(x)/2+x.z1_3+n2_3(x)-2*x.tbar_3
n0_4 = lambda x : 1-n1_3(x)-2*n2_3(x)+2*(x.tbar_3-x.z1_3-x.o_3-x.z2_3)

n2_5 = lambda x : x.tbar_4+x.o_4+x.z2_4
n1_5 = lambda x : n1_4(x)/2+x.z1_4+n2_4(x)-2*x.tbar_4
n0_5 = lambda x : 1-n1_4(x)-2*n2_4(x)+2*(x.tbar_4-x.z1_4-x.o_4-x.z2_4)


R_1 = lambda x : reps1(n1_0(x),n0_0(x),x.omid,x.zmid2,x.zmid1,1)
R_2 = lambda x : reps2(n1_1(x),n0_1(x),n2_1(x),x.tbar_1,x.o_1,x.z2_1,x.z1_1,1)
R_3 = lambda x : reps2(n1_2(x),n0_2(x),n2_2(x),x.tbar_2,x.o_2,x.z2_2,x.z1_2,1)
R_4 = lambda x : reps2(n1_3(x),n0_3(x),n2_3(x),x.tbar_3,x.o_3,x.z2_3,x.z1_3,1)
R_5 = lambda x : reps2(n1_4(x),n0_4(x),n2_4(x),x.tbar_4,x.o_4,x.z2_4,x.z1_4,1)


S_1 = lambda x : multiH(1,[n2_1(x),n2_1(x),n1_1(x),n1_1(x)])
S_2 = lambda x : multiH(1,[n2_2(x),n2_2(x),n1_2(x),n1_2(x)])
S_3 = lambda x : multiH(1,[n2_3(x),n2_3(x),n1_3(x),n1_3(x)])
S_4 = lambda x : multiH(1,[n2_4(x),n2_4(x),n1_4(x),n1_4(x)])
S_5 = lambda x : multiH(1,[n2_5(x),n2_5(x),n1_5(x),n1_5(x)])


L_1 = lambda x : S_1(x)-R_1(x)
L_2 = lambda x : S_2(x)-R_2(x)
L_3 = lambda x : S_3(x)-R_3(x)
L_4 = lambda x : S_4(x)-R_4(x)
L_5 = lambda x : S_5(x)-R_5(x)
L_6 = lambda x : S_5(x)/2

T_6 = lambda x : L_6(x)
T_5 = lambda x : 2*L_6(x) - R_5(x)
T_4 = lambda x : 2*L_5(x) - R_4(x) + R_5(x)
T_3 = lambda x : 2*L_4(x) - R_3(x) + R_4(x)   # missed R_(4)(x) in this line
T_2 = lambda x : 2*L_3(x) - R_2(x) + R_3(x)
T_1 = lambda x : 2*L_2(x) - R_1(x) + R_2(x)
T_0 = lambda x : 2*L_1(x) - 1


def time(x):
    x=set_vars(*x)
    max_of_T_i = max(T_6(x),T_5(x),T_4(x),T_3(x),T_2(x),T_1(x),T_0(x))
    return max_of_T_i

def memory(x):
    x=set_vars(*x)
    max_of_L_i = max(L_6(x),L_5(x),L_4(x),L_3(x),L_2(x),L_1(x))
    return max_of_L_i

constraints_lwe = [
    
    
   # { 'type' : 'ineq',   'fun' : lwe(lambda x : time(x)-memory(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_5(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n2_5(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_5(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_5(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : S_5(x))},

     
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_5(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : L_6(x))},


    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_5(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : T_6(x))},
#     { 'type' : 'ineq',   'fun' : lwe(lambda x : T_0(x))},  # this is negative

    
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_0(x)*.5-x.omid)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+n0_0(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_1+n2_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_1+n1_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_1-x.z1_1+n0_1(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_2+n2_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_2+n1_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_2-x.z1_2+n0_2(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_3+n2_3(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_3+n1_3(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_3-x.z1_3+n0_3(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_4+n2_4(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_4+n1_4(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_4-x.z1_4+n0_4(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_5(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_4(x)-R_5(x))},    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x)-R_4(x))},    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x)-R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x)-R_2(x))},
    

]

F6 = []    
for weight in range(25,26):
    w = weight*.02
    res = 100
    ress = 5000
    for j in range(90):
        start=[uniform(0,.009)]*19
        
        bounds=[(0.,0.4)]*19
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-12, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
               res = r
               ress = result
    if(res<memory(ress.x)):
        print('Memory exceeds time')
    print((round(w,4),round(res,4)))  
    F6.append((round(w,4),round(res,4)))

(0.5, 0.3296)
